
# 드라이브 연결 및 darknet 레포지토리 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/yolov5"
!git clone https://github.com/ultralytics/yolov5.git
!pip install -U -r yolov5/requirements.txt

[Errno 2] No such file or directory: '/content/drive/MyDrive/yolov5'
/content
Cloning into 'yolov5'...
remote: Enumerating objects: 13966, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 13966 (delta 107), reused 98 (delta 71), pack-reused 13809
Receiving objects: 100% (13966/13966), 13.58 MiB | 22.98 MiB/s, done.
Resolving deltas: 100% (9562/9562), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 15.9 MB/s 
     |████████████████████████████████| 793 kB 68.0 MB/s 
     |████████████████████████████████| 11.2 MB 62.2 MB/s 
     |████████████████████████████████| 3.2 MB 50.2 MB/s 
     |████████████████████████████████| 280 kB 77.1 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |██████████████████████████████  | 834.1 MB 1.2 MB/s eta 0:00:48tcmalloc: large alloc 1147494400 bytes == 0x65374000 @  0x

In [ ]:
%ls

drive/  sample_data/  yolov5/


In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
%cat Makefile

sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory
sed: can't read Makefile: No such file or directory
cat: Makefile: No such file or directory


In [ ]:
!make

make: *** No targets specified and no makefile found.  Stop.


In [ ]:
%ls

drive/  sample_data/  yolov5/


# 학습 데이터 준비

In [ ]:
%cd /content/drive/MyDrive/경운대 경진/train/img

/content/drive/MyDrive/경운대 경진/train/img


## 필요시 데이터 업로드

In [ ]:
import os
import random
def split_train_test(img_path):
  os.makedirs('../custom/',exist_ok=True)
  
  file_train = open('../custom/train.txt', 'w')
  file_valid = open('../custom/valid.txt', 'w')

  train_cnt = 0
  valid_cnt = 0

  for path in (img_path):
    print('path : ', path)
    path = path + '/img'
    img_ls = [i for i in os.listdir(path) if i[-3:]=='jpg']
    for idx in range(len(img_ls)):
      tar = 'dataset/'+path +'/'+img_ls[idx]
      print('tar : ', tar)
      if path == 'train/img':
        train_cnt+=1
        file_train.write(tar+"\n")
      else:
        valid_cnt+=1
        file_valid.write(tar+"\n")
  print('train_cnt : ', train_cnt)
  print('valid_cnt : ', valid_cnt)

In [ ]:
import numpy as np
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow as tf

train_img = tf.keras.preprocessing.image_dataset_from_directory(
    '../custom/train.txt',
    seed = 123,
    image_size = (128, 128, 3), # 사진 크기를 128대128로
    batch_size = 32, # 32개씩 데이터 처리
    subset = 'training', # training 데이터
)

val_img = tf.keras.preprocessing.image_dataset_from_directory(
    '../custom/valid.txt',
    seed = 123,
    image_size = (128, 128, 3),
    batch_size = 32,
    subset = 'validation', # val 데이터  
)

In [ ]:
%ls
img_path = os.listdir()
split_train_test(img_path)

train_cnt :  0
valid_cnt :  0


In [ ]:
%cd ..
%cp cfg/coco.data custom/yolov4-tiny.data 
%cp cfg/yolov4-tiny.cfg custom/yolov4-tiny.cfg
%cp  cfg/coco.names custom/yolov4-tiny.names

/content/gdrive/MyDrive/workplace/darknet


# custom 폴더 데이터 수정(예시)
자세한 정보는 darknet github 참고

## yolov4-tiny.data
classes= 4 \
train  = custom/train.txt \
valid  = custom/valid.txt \
names = custom/yolov4-tiny.names \
backup = backup/ \

## yolov4-tiny.names
car \
truck \
bus \
motocycle \


## yolov4-tiny.cfg
(test를 위해 max_batches를 낮게 설정함) \

subdivisions=16 \
max_batches = 2000 \
steps=1600,1800 \

[yolo] \
class = 4 \
yolo 노드 위의 [convolutional] \
filters=(class+5)*3 \
calc_anchors 결과에 따른 anchor값 수정

In [ ]:
!./darknet detector calc_anchors custom/yolov4-tiny.data custom/yolov4-tiny.cfg -num_of_clusters 6 -width 416 -height 416

/bin/bash: ./darknet: No such file or directory


# 윈도우 텍스트 유닉스용으로 변경

In [ ]:
!sudo apt install dos2unix
!dos2unix custom/train.txt
!dos2unix custom/valid.txt
!dos2unix custom/test.txt
!dos2unix custom/yolov4-tiny.cfg
!dos2unix custom/yolov4-tiny.data
!dos2unix custom/yolov4-tiny.names

yolov4-tiny pretrain 모델 다운로드

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

In [ ]:
!chmod +x ./darknet

# 학습진행

In [ ]:
!time ./darknet detector train custom/yolov4-tiny.data custom/yolov4-tiny.cfg yolov4-tiny.conv.29 -dont_show -map l tee train.log

# 모델 학습 결과 그래프 시각화

In [ ]:
def imShow(path): 
  import cv2 
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image, (3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

def upload():
  from google.colab import files
  uploaded = files.upload()
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)


In [ ]:
imShow('chart.png')

# test 이미지 내 Detection 실행 및 sample submission 작성

1. csv의 첫행은 선택한 클래스 명 기입
2. 두번째 행은 검출 정보의 column정보이며 \
'ImageID', 'LabelName','Conf', 'XMin', 'XMax', 'YMin', 'YMax'으로 제한함(순서의 변경도 허용하지 않음)

3. 세번째 행부터는 검출 결과를 column에 맞게 기입하며, **각 좌표는 정규화하여 기입**하여야한다.
 <br> (xmin, xmax, ymin, ymax = 실제 해당좌표, width = img_width, height = img_height)
$ XMin = xmin/width \\ $ \
$ XMax = xmax/width \\ $ \
$ YMin = ymin/height \\ $ \
$ YMax = ymax/height \\ $ \

In [ ]:
from darknet import *
import cv2
import os
import csv


f = open('sample_submission.csv','w', newline='')
wr = csv.writer(f)
wr.writerow(['ImageID', 'LabelName','Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

network, class_names, colors = load_network("custom/yolov4-tiny.cfg",  "custom/yolov4-tiny.data" ,"backup/yolov4-tiny_best.weights")

test_list = 'custom/test.txt'
rf = open(test_list, 'r')
lines = rf.readlines()
for line in lines:
    line = line.strip()
    frame = cv2.imread(line)
    frame =cv2.resize(frame,(network_width(network), network_height(network)))
    height, width, channel = frame.shape
    blk_img = make_image(network_width(network), network_height(network), 3)
    copy_image_from_bytes(blk_img, frame.tobytes())
    output = (detect_image(network, class_names, blk_img, thresh=.5, hier_thresh=.5, nms=.45))
    if output != []:
        for out in output:
            cls, conf, (x,y,w,h) = out
            xmin = max((x-(w/2)),0)
            xmax = min((x+(w/2)),width)
            ymin = max((y-(h/2)),0)
            ymax = min((y+(h/2)),height)
            XMin = xmin/width
            XMax = xmax/width
            YMin = ymin/height
            YMax = ymax/height
            det_data = [line.split('/')[-1],cls,float(conf)/100,XMin,XMax,YMin,YMax]
            wr.writerow(det_data)
rf.close()
f.close()
with open('sample_submission.csv', newline='') as f:
    rd = csv.reader(f)
    for row in rd:
        print(', '.join(row))